<a href="https://colab.research.google.com/github/tompollard/tableone/blob/master/tableone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demonstrating the `tableone` package

In research papers, it is common for the first table ("Table 1") to display summary statistics of the study data. The `tableone` package is used to create this table. For an introduction to basic statistical reporting in biomedical journals, we recommend reading the [SAMPL Guidelines](http://www.equator-network.org/wp-content/uploads/2013/07/SAMPL-Guidelines-6-27-13.pdf). For more reading on accurate reporting in health research, visit the [EQUATOR Network](http://www.equator-network.org/).

## A note for users of `tableone`

While we have tried to use best practices in creating this package, automation of even basic statistical tasks can be unsound if done without supervision. We encourage use of `tableone` alongside other methods of descriptive statistics and, in particular, visualization to ensure appropriate data handling. 

It is beyond the scope of our documentation to provide detailed guidance on summary statistics, but as a primer we provide some considerations for choosing parameters when creating a summary table at: http://tableone.readthedocs.io/en/latest/bestpractice.html. 

*Guidance should be sought from a statistician when using `tableone` for a research study, especially prior to submitting the study for publication*.

## Suggested citation
If you use tableone in your study, please cite the following paper:

> Tom J Pollard, Alistair E W Johnson, Jesse D Raffa, Roger G Mark; *tableone*: An open source Python package for producing summary statistics for research papers, *JAMIA Open*, Volume 1, Issue 1, 1 July 2018, Pages 26–31, https://doi.org/10.1093/jamiaopen/ooy012

Download the BibTex file from: https://academic.oup.com/jamiaopen/downloadcitation/5001910?format=bibtex

## Installation

To install the package with pip, run the following command in your terminal: ``pip install tableone``. To install the package with Conda, run: ``conda install -c conda-forge tableone``. For more detailed installation instructions, refer to the [documentation](http://tableone.readthedocs.io/en/latest/install.html).

## Importing libraries

Before using the `tableone` package, we need to import it. We will also import `pandas` for loading our sample dataset and `matplotlib` for creating plots.

In [0]:
# import numerical libraries
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
# import tableone
try:
    from tableone import TableOne
except ModuleNotFoundError:
    # install on Colab
    !pip install tableone
    import tableone

## Loading sample data

We begin by loading the data that we would like to summarize into a Pandas DataFrame. 
- Variables are in columns
- Encounters/observations are in rows.

In [0]:
# load sample data into a pandas dataframe
url="https://raw.githubusercontent.com/tompollard/tableone/master/data/pn2012_demo.csv"
data=pd.read_csv(url)

In [0]:
data.head()

## Example 1: Simple summary of data with Table 1

In this example we provide summary statistics across all of the data.

In [0]:
# view the tableone docstring
TableOne??

In [0]:
# create an instance of TableOne with the input arguments
# firstly, with no grouping variable
overall_table = TableOne(data, label_suffix=True)

In [0]:
# view first 10 rows of tableone
overall_table

In [0]:
# the pd.DataFrame object can be accessed using the `tableone` attribute
type(overall_table.tableone)

**Summary of the table**:
- the first row ('`n`') displays a count of the encounters/observations in the input data.
- the '`Missing`' column displays a count of the null values for the particular variable.
- if categorical variables are not defined in the arguments, they are detected automatically.
- continuous variables (e.g. '`age`') are summarized by '`mean (std)`'.
- categorical variables (e.g. '`ascites`') are summarized by '`n (% of non-null values)`'.
- if `label_suffix=True`, "mean (SD); n (%);" etc are appended to the row label.

## Displaying the table in custom formats

The tableone object includes a `tabulate` method, that makes use of the [tabulate](https://pypi.org/project/tabulate/) package to display the table in custom output formats. Supported table formats include: "github", "grid", "fancy_grid", "rst", "html", "latex", and "latex_raw". See the tabulate package for more formats.

In [0]:
print(overall_table.tabulate(tablefmt = "fancy_grid"))

## Exporting your table to LaTex

To export your table in LaTex (for example, to add to your document on [Overleaf.com](https://www.overleaf.com/)), it's simple with the `tabulate` method. Just copy and paste the output below.

In [0]:
print(overall_table.tabulate(tablefmt = "latex"))

## Exploring the warning raised by Hartigan's Dip Test

Hartigan's Dip Test is a test for multimodality. The test has suggested that the `Age`, `SysABP`, and `Height` distributions may be multimodal. We'll plot the distributions here.

In [0]:
data[['Age','SysABP','Height']].dropna().plot.kde(figsize=[12,8])
plt.legend(['Age (years)', 'SysABP (mmHg)', 'Height (cm)'])
plt.xlim([-30,250])

## Exploring the warning raised by Tukey's rule

Tukey's rule has found far outliers in Height, so we'll look at this in a boxplot

In [0]:
data[['Age','Height','SysABP']].boxplot(whis=3)
plt.show()

In both cases it seems that there are values that may need to be taken into account when calculating the summary statistics. For `SysABP`, a clearly bimodal distribution, the researcher will need to decide how to handle the peak at ~0, perhaps by cleaning the data and/or describing the issue in the summary table. For `Height`, the researcher may choose to report median, rather than mean.

## Example 2: Table 1 without stratification

In this example we provide summary statistics across all of the data, specifying columns, categorical variables, and non-normal variables.

In [0]:
# columns to summarize
columns = ['Age', 'SysABP', 'Height', 'Weight', 'ICU', 'death']

# columns containing categorical variables
categorical = ['ICU', 'death']

# non-normal variables
nonnormal = ['Age']

# limit the binary variable "death" to a single row
limit = {"death": 1}

# set the order of the categorical variables
order = {"ICU": ["MICU", "SICU", "CSRU", "CCU"],
         "death": ["1"]}

# alternative labels
labels={'death': 'Mortality'}

# set decimal places for age to 0
decimals = {"Age": 0}

# create tableone with the input arguments
mytable = TableOne(data, columns=columns, categorical=categorical, 
                   nonnormal=nonnormal, rename=labels, label_suffix=True, 
                   decimals=decimals, limit=limit, order=order)

mytable

**Summary of the table**:

- variables are explicitly defined in the input arguments.
- the variables are displayed in the same order as the `columns` argument.
- the `limit` argument specifies that only a 1 value should be shown for death.
- the order of categorical values is defined in the optional `order` argument.
- `nonnormal` continuous variables are summarized by '`median [Q1,Q3]`' instead of `mean (SD)`.
- 'sex' is shown as 'gender and 'trt' is shown as 'treatment', as specified in the `rename` argument.

## Example 3: Table 1 with stratification

In this example, we group data across a categorical variable.

In [0]:
# optionally, a categorical variable for stratification
groupby = ['death']

# rename the death column
labels={'death': 'Mortality'}

In [0]:
# create an instance of TableOne with the input arguments
grouped_table = TableOne(data, columns, categorical, groupby, 
                         nonnormal, rename=labels, label_suffix=True)

In [0]:
# view first 10 rows of tableone
grouped_table

**Summary of the table**:
- data is summarized across the groups specified in the `groupby` argument.
- as before, the summary statistics are either '`mean (std)`', '`median [Q1,Q3]`', or '`n (% of non-null values)`'.

## Computing p-values

We can run a test to compute p values by setting the ``pval`` argument to True.

In [0]:
# create grouped_table with p values
grouped_table = TableOne(data, columns, categorical, groupby, 
                         nonnormal, label_suffix=True, pval = True)

In [0]:
# view first 10 rows of tableone
grouped_table

**Summary of the table**:
- the '`Test`' column displays the name of the test used to compare the groups.
- the '`p-value`' column displays the p value generated by the test in the '`Test`' column, to 3 decimal places.

## Exporting the table to file (LaTeX, CSV, etc)

Tables can be exported to file in various formats, including:
- LaTeX
- CSV
- HTML

To export the table, call the relevant `to_<format>()` method on the DataFrame.

In [0]:
# Save to Excel
fn = 'tableone.xlsx'
grouped_table.to_excel(fn)

In [0]:
# Save table to LaTeX
fn = 'tableone.tex'
grouped_table.to_latex(fn)

In [0]:
# Save table to HTML
fn2 = 'tableone.html'
grouped_table.to_html(fn2)